<a href="https://colab.research.google.com/github/christophermalone/DSCI325/blob/main/Final_MNSchoolDisciplinePython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final: MN School Discipline - Python

In [1]:
import pandas as pd
import numpy as np

## Preparing the MN Schools Discipline Data

In [101]:
Discipline = pd.read_csv("/content/sample_data/MNSchools_Discipline.csv", dtype={'Year': str, 'CountyFIPS': str, 'DistrictNumber': str, 'DistrictType': str})

In [102]:
Discipline.head()

,Year,CountyFIPS,DistrictNumber,DistrictType,DistrictName,AmericanIndian,AsianPI,Black,Hispanic,White,MultiRace,Total
0,18-19,1,1,1,AITKIN PUBLIC SCHOOL DISTRICT,NaN,NaN,5.0,NaN,61.0,NaN,NaN
1,18-19,27,1,3,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0
2,18-19,1,2,1,HILL CITY PUBLIC SCHOOL DISTRICT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18-19,1,4,1,MCGREGOR PUBLIC SCHOOL DISTRICT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18-19,19,6,3,South St. Paul Public School Dist,5.0,NaN,78.0,127.0,167.0,52.0,NaN


In [103]:
pip install dfply

In [104]:
from dfply import *

In [170]:
Discipline_Schools= (
                Discipline
                 >> filter_by(X.Total.notnull())
               )

Discipline_Schools.head(20)

,Year,CountyFIPS,DistrictNumber,DistrictType,DistrictName,AmericanIndian,AsianPI,Black,Hispanic,White,MultiRace,Total
1,18-19,27,1,3,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0
5,18-19,2,11,1,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0
6,18-19,2,12,1,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0
9,18-19,2,15,1,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0
10,18-19,2,16,1,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0
53,18-19,19,191,1,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0
57,18-19,19,196,1,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0
82,18-19,27,279,1,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0
84,18-19,27,281,1,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0
86,18-19,27,283,1,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0


In [171]:
Discipline_Schools_withMNSchoolID= (
                             Discipline_Schools
                             >> mutate(DistrictNumber_Prefix= case_when(
                                                                          [X.DistrictNumber.str.len() == 1, '000'],
                                                                          [X.DistrictNumber.str.len() == 2, '00'],
                                                                          [X.DistrictNumber.str.len() == 3, '0'],
                                                                          [X.DistrictNumber.str.len() == 4, '']
                                                                       )
                                      )
                             >> mutate(DistrictNumber_Fixed = X.DistrictNumber_Prefix+X.DistrictNumber)
                             >> mutate(DistrictType_Prefix= case_when(
                                                                        [X.DistrictType.str.len() == 1, '0'],
                                                                        [X.DistrictType.str.len() == 2, '']
                                                                     )
                                     )
                             >> mutate(DistrictType_Fixed = X.DistrictType_Prefix+X.DistrictType) 
                             >> mutate(MNSchoolID = X.DistrictNumber_Fixed+'-'+X.DistrictType_Fixed) 
                             >> select(X.DistrictName, X.AmericanIndian, X.AsianPI, X.Black, X.Hispanic, X.White, X.MultiRace, X.Total, X.MNSchoolID)
                           )

Discipline_Schools_withMNSchoolID.head(20)

,DistrictName,AmericanIndian,AsianPI,Black,Hispanic,White,MultiRace,Total,MNSchoolID
1,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0,0001-03
5,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0,0011-01
6,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0,0012-01
9,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0,0015-01
10,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0,0016-01
53,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0,0191-01
57,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0,0196-01
82,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0,0279-01
84,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0,0281-01
86,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0,0283-01


## Preparing MN School Information

In [172]:
School_Info= pd.read_csv("/content/sample_data/MNSchools_Information.csv", dtype = 'str')

In [173]:
School_Info.head()

,DistrictNumber,DistrictType,SchoolNumber,MNSchoolID,SchoolName,Address,City,State,Zipcode,County,Grades,NCESID,StateOrganizationId,ACTID
0,1,1,0,0001-01,Aitkin Public School District,306 2nd St NW,Aitkin,MN,56431-1289,"Aitkin County, MN",NaN,2702760,10001000000,247002
1,1,1,1,0001-01-001,Aitkin Secondary School,306 2nd St NW,Aitkin,MN,56431,"Aitkin County, MN",7-12,2.70276E+11,10001001000,240020
2,1,1,2,0001-01-002,Rippleside Elementary,225 2nd Ave SW,Aitkin,MN,56431,"Aitkin County, MN",EC-6,2.70276E+11,10001002000,NaN
3,1,1,6,0001-01-006,Aitkin Alternative Learning Program,12 7th St NW,Aitkin,MN,56431,"Aitkin County, MN",7-12,2.70276E+11,10001006000,NaN
4,1,1,8,0001-01-008,Aitkin ESY,306 2nd St NW,Aitkin,MN,56431,"Aitkin County, MN",1-12,2702760,10001008000,NaN


## JOIN Discipline_Schools_withMNSchoolID with School_Info

In [177]:
Discipline_Schools_withMNSchoolID_withNCESID = (
                                          Discipline_Schools_withMNSchoolID
                                          >> left_join(School_Info, by='MNSchoolID')
                                          >> select(X.DistrictName, X.AmericanIndian, X.AsianPI, X.Black, X.Hispanic, X.White, X.MultiRace, X.Total, X.MNSchoolID, X.NCESID)
                                        )

Discipline_Schools_withMNSchoolID_withNCESID.head(20)

,DistrictName,AmericanIndian,AsianPI,Black,Hispanic,White,MultiRace,Total,MNSchoolID,NCESID
0,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0,0001-03,2721240
1,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0,0011-01,2703180
2,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0,0012-01,2708100
3,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0,0015-01,2733540
4,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0,0016-01,2733330
5,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0,0191-01,2707290
6,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0,0196-01,2732390
7,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0,0279-01,2725200
8,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0,0281-01,2731780
9,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0,0283-01,2733780


## Preparing Census Data

In [175]:
Census_Data= pd.read_csv("/content/sample_data/ACS2019_CensusData_SchoolLevel.csv", dtype={'NCESID': str} )

## JOIN Discipline_Schools_withMNSchoolID_withNCESID to Census_Data

In [180]:
Discipline_CensusData_AllColumns = (
                         Discipline_Schools_withMNSchoolID_withNCESID
                         >> left_join(Census_Data, by='NCESID')
                        )

Discipline_CensusData_AllColumns.head(20)

,DistrictName,AmericanIndian,AsianPI,Black,Hispanic_x,White,MultiRace,Total,MNSchoolID,NCESID,GeographicUnit,SchoolDistrictName,Source,Year,Population,HouseholdSize,WhiteNH,BlackNH,AmerIndianNH,AsianPINH,MultiRaceNH,Hispanic_y,NonHispanic,LessThanHS,BachelorsOrMore,NumberEmployed,CommuteTime,IncomePerCapita,MedianHouseholdIncome,PovertyPct,HomeownerPct
0,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0,0001-03,2721240,SCHLDIST,Minneapolis Public School District Minnesota,ACS 2015-2019,2019,420280,2.28,252154,79434,4804,24935,17492,40432,379848,28146,141463,238155,23.22,38808,62582,0.191,0.472
1,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0,0011-01,2703180,SCHLDIST,Anoka-Hennepin Public School District Minnesota,ACS 2015-2019,2019,241076,2.76,193271,17803,649,13102,6345,9630,231446,8578,51513,130374,27.19,37380,85112,0.054,0.812
2,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0,0012-01,2708100,SCHLDIST,Centennial Public School District Minnesota,ACS 2015-2019,2019,32855,2.82,28417,964,252,1187,1178,857,31998,968,8627,17968,26.39,40482,98067,0.033,0.883
3,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0,0015-01,2733540,SCHLDIST,St. Francis Public School District Minnesota,ACS 2015-2019,2019,33061,2.86,30314,233,172,829,755,742,32319,1280,5116,18584,34.94,37434,92107,0.047,0.931
4,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0,0016-01,2733330,SCHLDIST,Spring Lake Park Public Schools Minnesota,ACS 2015-2019,2019,31804,2.60,23332,1808,373,2326,1247,2718,29086,1690,6643,16573,25.17,37921,73207,0.079,0.785
5,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0,0191-01,2707290,SCHLDIST,Burnsville Public School District Minnesota,ACS 2015-2019,2019,68261,2.51,47844,8645,117,4197,2353,4839,63422,3180,18138,38444,24.29,36919,78002,0.079,0.680
6,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0,0196-01,2732390,SCHLDIST,Rosemount-Apple Valley-Eagan Minnesota,ACS 2015-2019,2019,153845,2.69,118252,11691,150,9938,5029,8590,145255,4045,49974,84996,25.60,44721,96642,0.050,0.784
7,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0,0279-01,2725200,SCHLDIST,Osseo Public School District Minnesota,ACS 2015-2019,2019,146643,2.68,92406,24894,464,18159,4743,5611,141032,6211,42044,79064,25.78,41284,88580,0.062,0.778
8,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0,0281-01,2731780,SCHLDIST,Robbinsdale Public School District Minnesota,ACS 2015-2019,2019,102910,2.39,70119,16432,336,4287,3609,7926,94984,5083,29068,55742,24.10,38775,69714,0.091,0.655
9,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0,0283-01,2733780,SCHLDIST,St. Louis Park Public School District Minnesota,ACS 2015-2019,2019,46804,2.08,37892,3132,212,2040,1294,2055,44749,1116,21324,28656,21.77,49146,81246,0.066,0.566


## Doing Calucations for Chi-Square Measure

In [187]:
Discipline_CensusData = (
                          Discipline_CensusData_AllColumns
                          >> mutate(TotalCensus = X.WhiteNH + X.BlackNH + X.AmerIndianNH + X.AsianPINH + X.MultiRaceNH + X.Hispanic_y)
                          >> select(X.DistrictName, X.AmericanIndian, X.AsianPI, X.Black, X.Hispanic_x, X.White, X.MultiRace, X.Total, X.WhiteNH, X.BlackNH, X.AmerIndianNH, X.AsianPINH, X.MultiRaceNH, X.Hispanic_y, X.TotalCensus)
                         )
 
Discipline_CensusData.head(20)

,DistrictName,AmericanIndian,AsianPI,Black,Hispanic_x,White,MultiRace,Total,WhiteNH,BlackNH,AmerIndianNH,AsianPINH,MultiRaceNH,Hispanic_y,TotalCensus
0,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0,252154,79434,4804,24935,17492,40432,419251
1,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0,193271,17803,649,13102,6345,9630,240800
2,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0,28417,964,252,1187,1178,857,32855
3,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0,30314,233,172,829,755,742,33045
4,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0,23332,1808,373,2326,1247,2718,31804
5,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0,47844,8645,117,4197,2353,4839,67995
6,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0,118252,11691,150,9938,5029,8590,153650
7,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0,92406,24894,464,18159,4743,5611,146277
8,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0,70119,16432,336,4287,3609,7926,102709
9,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0,37892,3132,212,2040,1294,2055,46625


In [189]:
Discipline_CensusData_withProp =(
                        Discipline_CensusData
                        >> mutate(
                                    WhiteProp = X.WhiteNH / X.TotalCensus,
                                    BlackProp = X.BlackNH / X.TotalCensus,
                                    AmerIndianProp = X.AmerIndianNH / X.TotalCensus,
                                    AsianPIProp = X.AsianPINH / X.TotalCensus,
                                    MultiRaceProp = X.MultiRaceNH / X.TotalCensus,
                                    HispanicProp = X.Hispanic_y / X.TotalCensus
                                 )
                        >> select(X.DistrictName, X.AmericanIndian, X.AsianPI, X.Black, X.Hispanic_x, X.White, X.MultiRace, X.Total,X.WhiteProp, X.BlackProp, X.AmerIndianProp, X.AsianPIProp,X.MultiRaceProp, X.HispanicProp )
                       )

Discipline_CensusData_withProp.head(20)

,DistrictName,AmericanIndian,AsianPI,Black,Hispanic_x,White,MultiRace,Total,WhiteProp,BlackProp,AmerIndianProp,AsianPIProp,MultiRaceProp,HispanicProp
0,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0,0.601439,0.189466,0.011459,0.059475,0.041722,0.096439
1,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0,0.802620,0.073933,0.002695,0.054410,0.026350,0.039992
2,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0,0.864922,0.029341,0.007670,0.036128,0.035855,0.026084
3,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0,0.917355,0.007051,0.005205,0.025087,0.022848,0.022454
4,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0,0.733618,0.056848,0.011728,0.073135,0.039209,0.085461
5,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0,0.703640,0.127142,0.001721,0.061725,0.034605,0.071167
6,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0,0.769619,0.076089,0.000976,0.064679,0.032730,0.055906
7,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0,0.631719,0.170184,0.003172,0.124141,0.032425,0.038359
8,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0,0.682696,0.159986,0.003271,0.041739,0.035138,0.077169
9,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0,0.812697,0.067174,0.004547,0.043753,0.027753,0.044075


In [191]:
Discipline_CensusData_withExpected = (
                                    Discipline_CensusData_withProp
                                    >> mutate(
                                                WhiteExp = X.WhiteProp * X.Total,
                                                BlackExp = X.BlackProp * X.Total,
                                                AmerIndianExp = X.AmerIndianProp * X.Total,
                                                AsianPIExp = X.AsianPIProp * X.Total,
                                                MultiRaceExp = X.MultiRaceProp * X.Total,
                                                HispanicExp = X.HispanicProp * X.Total
                                             )
                                    >> select(X.DistrictName, X.AmericanIndian, X.AsianPI, X.Black, X.Hispanic_x, X.White, X.MultiRace, X.Total, X.WhiteExp, X.BlackExp, X.AmerIndianExp, X.AsianPIExp,X.MultiRaceExp, X.HispanicExp)
                                    )

Discipline_CensusData_withExpected.head(20)


,DistrictName,AmericanIndian,AsianPI,Black,Hispanic_x,White,MultiRace,Total,WhiteExp,BlackExp,AmerIndianExp,AsianPIExp,MultiRaceExp,HispanicExp
0,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0,3902.137746,1229.258349,74.342940,385.874524,270.692487,625.693954
1,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0,1989.696051,183.279223,6.681358,134.883131,65.320826,99.139410
2,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0,300.992726,10.210683,2.669183,12.572698,12.477370,9.077340
3,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0,122.925586,0.944833,0.697473,3.361658,3.061583,3.008867
4,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0,428.433153,33.199346,6.849201,42.711106,22.898000,49.909194
5,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0,546.728259,98.789102,1.336995,47.960424,26.888462,55.296757
6,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0,625.700462,61.859961,0.793687,52.584406,26.609678,45.451806
7,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0,795.966283,214.431797,3.996801,156.417892,40.855227,48.332000
8,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0,632.176284,148.147017,3.029296,38.650576,32.537889,71.458937
9,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0,227.555174,18.808794,1.273137,12.250938,7.770938,12.341019


In [192]:
Discipline_CensusData_ChiSquare= (
                                    Discipline_CensusData_withExpected
                                    >> mutate(ChiSquare = 
                                              (((X.AmericanIndian - X.AmerIndianExp)**2)/X.AmerIndianExp) +
                                              (((X.AsianPI - X.AsianPIExp)**2)/X.AsianPIExp) +
                                              (((X.Black - X.BlackExp)**2)/X.BlackExp)+ 
                                              (((X.Hispanic_x - X.HispanicExp)**2)/X.HispanicExp) +
                                              (((X.White - X.WhiteExp)**2)/X.WhiteExp) +
                                              (((X.MultiRace - X.MultiRaceExp)**2)/X.MultiRaceExp)
                                              )
                                            
                                )
Discipline_CensusData_ChiSquare.head(20)

,DistrictName,AmericanIndian,AsianPI,Black,Hispanic_x,White,MultiRace,Total,WhiteExp,BlackExp,AmerIndianExp,AsianPIExp,MultiRaceExp,HispanicExp,ChiSquare
0,Minneapolis Public School District,327.0,50.0,4845.0,470.0,520.0,276.0,6488.0,3902.137746,1229.258349,74.342940,385.874524,270.692487,625.693954,14756.642066
1,ANOKA-HENNEPIN PUBLIC SCHOOL DIST.,44.0,42.0,781.0,197.0,1109.0,306.0,2479.0,1989.696051,183.279223,6.681358,134.883131,65.320826,99.139410,3594.944112
2,CENTENNIAL PUBLIC SCHOOL DISTRICT,7.0,8.0,43.0,38.0,224.0,28.0,348.0,300.992726,10.210683,2.669183,12.572698,12.477370,9.077340,245.145817
3,St. Francis Area Schools,5.0,6.0,9.0,8.0,99.0,7.0,134.0,122.925586,0.944833,0.697473,3.361658,3.061583,3.008867,115.288552
4,SPRING LAKE PARK PUBLIC SCHOOLS,16.0,19.0,179.0,99.0,207.0,64.0,584.0,428.433153,33.199346,6.849201,42.711106,22.898000,49.909194,902.208290
5,BURNSVILLE PUBLIC SCHOOL DISTRICT,24.0,18.0,334.0,201.0,106.0,94.0,777.0,546.728259,98.789102,1.336995,47.960424,26.888462,55.296757,1869.595822
6,ROSEMOUNT-APPLE VALLEY-EAGAN,7.0,23.0,247.0,114.0,342.0,80.0,813.0,625.700462,61.859961,0.793687,52.584406,26.609678,45.451806,958.417083
7,OSSEO PUBLIC SCHOOL DISTRICT,13.0,18.0,867.0,90.0,154.0,118.0,1260.0,795.966283,214.431797,3.996801,156.417892,40.855227,48.332000,2828.046677
8,ROBBINSDALE PUBLIC SCHOOL DISTRICT,19.0,5.0,660.0,73.0,64.0,105.0,926.0,632.176284,148.147017,3.029296,38.650576,32.537889,71.458937,2554.028109
9,ST. LOUIS PARK PUBLIC SCHOOL DIST.,10.0,5.0,141.0,31.0,56.0,37.0,280.0,227.555174,18.808794,1.273137,12.250938,7.770938,12.341019,1125.413281


In [185]:
Discipline_Outcomes = (
                        Discipline_CensusData_ChiSquare
                        >> arrange(X.ChiSquare)
                        >> mutate(Rank = X.ChiSquare.rank())
                        >> select(X.DistrictName, X.ChiSquare,X.Rank)
                      )
Discipline_Outcomes = Discipline_Outcomes.round(0)
print(Discipline_Outcomes.to_string(index=False))

                       DistrictName  ChiSquare  Rank
 FOREST LAKE PUBLIC SCHOOL DISTRICT       87.0   1.0
           St. Francis Area Schools      115.0   2.0
  CENTENNIAL PUBLIC SCHOOL DISTRICT      245.0   3.0
    SHAKOPEE PUBLIC SCHOOL DISTRICT      303.0   4.0
     PRIOR LAKE-SAVAGE AREA SCHOOLS      574.0   5.0
    SPRING LAKE PARK PUBLIC SCHOOLS      902.0   6.0
       ROSEMOUNT-APPLE VALLEY-EAGAN      958.0   7.0
   North St. Paul-Maplewood Oakdale     1038.0   8.0
 ST. LOUIS PARK PUBLIC SCHOOL DIST.     1125.0   9.0
  BURNSVILLE PUBLIC SCHOOL DISTRICT     1870.0  10.0
   ROCHESTER PUBLIC SCHOOL DISTRICT     1921.0  11.0
 ROBBINSDALE PUBLIC SCHOOL DISTRICT     2554.0  12.0
       OSSEO PUBLIC SCHOOL DISTRICT     2828.0  13.0
 ANOKA-HENNEPIN PUBLIC SCHOOL DIST.     3595.0  14.0
    ST. PAUL PUBLIC SCHOOL DISTRICT    10650.0  15.0
 Minneapolis Public School District    14757.0  16.0
